In [4]:
#%pip install pyspark

In [5]:
#%pip install google-cloud-storage

TO DO:

- clean up code
- might want to not point to the latest gsc connector but an actual version for compatability
- write post on it

In [ ]:
from pyspark.sql import SparkSession
import os

# .config("spark.hadoop.gooogle.cloud.auth.null.credential.enabled", "true") \
# .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
#.config("spark.hadoop.google.cloud.auth.type","APPLICATION_DEFAULT") \
#.config("spark.jars.packages","com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.26") \

"""
This works for spark to write directly to gcs in parquet; if i try to go via ivy with spark.jars.packages, it blows up and doesnt work
"""

spark = SparkSession.builder \
    .appName("GCSTest") \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
    .getOrCreate()


gcs_path = f"gs://{os.getenv("GCS_BUCKET")}/data_output2"

# Create a DataFrame with dummy data
data = [("Alice", 25), ("Bob", 30), ("Cathy", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
#print(df)
df.write.mode("overwrite").parquet(gcs_path)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 06:24:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
## This works using a google service account
from pyspark.sql import SparkSession
import os
catalog_name = "iceberg"

def get_spark_instance_via_gsa(catalog_name) -> SparkSession:

    spark_version = "3.5"
    scala_version = "2.12"
    iceberg_version = "1.7.0"

    import os
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import current_date, rand, floor, expr

    #catalog_name = "iceberg"
    #warehouse_path = "./icehouse"

    warehouse_path = f"gs://{os.getenv("GCS_BUCKET")}/icehouse"
    #print(gcs_path)

    return SparkSession.builder \
        .appName("local_iceberg_example") \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
        .config(f"spark.sql.catalog.{catalog_name}.type", "hadoop") \
        .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
        .config("spark.sql.warehouse.dir", warehouse_path) \
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version},com.google.guava:guava:30.1.1-jre") \
        .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
        .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", os.getenv("GOOGLE_APPLICATION_CREDENTIALS")) \
        .getOrCreate()

In [ ]:
spark = get_spark_instance_via_gsa(catalog_name)

In [3]:
# Create a DataFrame with dummy data
data = [("Alice", 25), ("Bob", 30), ("Cathy", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
print(df)


DataFrame[Name: string, Age: bigint]


In [4]:
namespace = "dummy_ns"

## not needed
#spark.sql(f"create namespace {namespace}")

In [ ]:
table_name = "dummy_data5"

df.writeTo(f"{catalog_name}.{namespace}.{table_name}") \
    .using("iceberg") \
    .createOrReplace()